### Notebook 2.1: Preprocessing - Missing Values
**Author:<br>
Tashi T. Gurung**<br>
**hseb.tashi@gmail.com**

### Project Objective

The primary aim of this project is to predict and understand the factors contributing to the failure of water points located throughout Tanzania.

### Background

Tanzania faces a critical challenge with nearly 50% of its population lacking access to safe drinking water. Among the various sources of water, the country heavily relies on approximately 60,000 water points, predominantly pumps. These water points are spread across diverse geographic regions, making data collection on their conditions a formidable task. Despite efforts involving local agencies and mobile-based crowd-sourcing initiatives, obtaining reliable data remains elusive.

The absence of high-quality data poses several issues, particularly for stakeholders like the Tanzanian Government, notably the Ministry of Water. Consequences encompass increased maintenance expenditures and the myriad of challenges faced by communities when their water access is compromised.

While the development of improved data collection infrastructure is a long-term goal, this project aspires to deliver valuable insights through predictive models, analyses, and uncovering the determinants of water point failure. Our objective is not solely limited to forecasting failures but also to shed light on the underlying causes. These insights will prove instrumental in optimizing resource allocation to maximize water access for individuals and communities. In the long run, this project will aid stakeholders in project planning and inform local, regional, and national-level policy formulation.


---

### Import Libraries

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings

# Filter out FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [28]:
# import json file with desired data type information 
json_file_path = '../data/processed/data_types.json'

# Read and load the JSON file into a dictionary
with open(json_file_path, 'r') as json_file:
    data_types_dict = json.load(json_file)

### Import Datasets

In [29]:
df = pd.read_csv('../data/processed/post_eda_data.csv', dtype=data_types_dict)
df.head(2)

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,Lake Nyasa,Mnyusi B,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,Lake Victoria,Nyamara,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


---

### Missing Values

**Numerical Columns**

In [30]:
# get a list of numerical features
numerical_features = df.drop(columns = ['status_group'])\
                            .select_dtypes(include=['int64', 'float64'])\
                                .columns\
                                    .to_list()

# get a series of numerical columns with 
columns_with_zeros: pd.Series = df[numerical_features].apply(lambda col: (col == 0).sum())

# Display columns with a significant number of zeros

print("Absolute Number:")
print(columns_with_zeros[columns_with_zeros > 0])

print("\nIn PERCENTAGE %:")
print(columns_with_zeros[columns_with_zeros > 0]/len(df) * 100)

Absolute Number:
amount_tsh           41639
gps_height           20438
longitude             1812
population           21381
construction_year    20709
dtype: int64

In PERCENTAGE %:
amount_tsh           70.099327
gps_height           34.407407
longitude             3.050505
population           35.994949
construction_year    34.863636
dtype: float64


**Observation:** <br>
Potential explanation for zero values

1. amount_tsh: represents waterpoints with no water available.
2. gps_height: zero represents sea-level
3. longitude: 0 places the location in the Atlantic Ocean, so definitely missing value
4. population: either no people live near the pump, or it's missing value
5. construction_year: definitely missing value

---

### Lets take a closer look at these 5 features

### 1. feature: amount_tsh

**Question:** <br>
What percentage of **functional** water pumps have no water i.e. amount_tsh = 0?

In [31]:
# number of functional pumps that have no water
functional_zero_tsh_mask =(df['status_group'] == 'functional') & (df['amount_tsh'] == 0)
functional_zero_tsh_len = len(df[functional_zero_tsh_mask]) 
print(f" - Number of functional pumps with amount_tsh as zero: {functional_zero_tsh_len}")

# total number of functional pumps
total_functional_pumps = len(df[df['status_group'] == 'functional'])
print(f" - Number of functional pumps is: {total_functional_pumps}")

# percentage of functional pumps with no water
ans = round(functional_zero_tsh_len/total_functional_pumps, 4) * 100
print(f"ANSWER: percentage of functional pumps with feature:'amount_tsh'== 0 is {ans} %")


 - Number of functional pumps with amount_tsh as zero: 19706
 - Number of functional pumps is: 32259
ANSWER: percentage of functional pumps with feature:'amount_tsh'== 0 is 61.09 %


**Answer:** <br>
- ~61% of functional pumps have an 'amount_tsh' value of zero <br>
<br>

**Possibilities:** <br>
 - missing value
 - might be genuine, pump might be functional, but there might be no water

**Action:** <br>

**In the future:**
- We might do a segmentation analysis. For example, we can check if functional pumps with zero 'amount_tsh' are more common in certain regions, subvillages, or based on other categorical variables. This can provide insight into whether these zeros are valid or likely missing data.
<br>

**For now:**
- treat 0s as missing value
- impute missing values with median

In [32]:
def impute_nan_by_status_group(feature_name, status, aggregation_method):
    aggregator_series = df.groupby('status_group')[feature_name].aggregate(aggregation_method)
    
    boolean_mask = (df['status_group'] == status) & (df[feature_name].isna())
    df.loc[boolean_mask, feature_name] = aggregator_series[status]
    
    print(f'Replaced nulls with {aggregator_series[status]} for {status}')
    return df

In [33]:
# replace 0's with nan
df['amount_tsh'].replace(0, np.nan, inplace=True)

In [34]:
# Making separate calls to the function within a loop
for status in df['status_group'].unique():
    df = impute_nan_by_status_group(feature_name = 'amount_tsh', status=status, aggregation_method='median')

Replaced nulls with 250.0 for functional
Replaced nulls with 200.0 for non functional
Replaced nulls with 300.0 for functional needs repair


In [35]:
# sanity check
df['amount_tsh'].isna().sum()

0

---

### 2. feature: gps_height

**Observation:** <br>
- zero represents sea-level
- the highest point in Tanzania is 5895 m (19,341 ft)
- the lowest is floor of Lake Tanganyika, which goes down to 352 m (1,115 ft) below sea level at its deepest.
- - the lakes surface is above sea level.
- zero could be valid
- - but given the high proportion of zeros, it might also contain  missing data 
<br>

**Assumption:** <br>
- We will assume it to be missing data

**Action:** <br>

**In the future:**
- look more into it, someone with knowledge of Tanzania's topography might be instrumental

**For now:**
- impute missing values with median

In [36]:
df['gps_height'].replace(0, np.nan, inplace=True)

for status in df['status_group'].unique():
    df = impute_nan_by_status_group(feature_name = 'gps_height', status=status, aggregation_method='median')

Replaced nulls with 1249.0 for functional
Replaced nulls with 997.0 for non functional
Replaced nulls with 1159.5 for functional needs repair


In [37]:
# sanity check
df['gps_height'].isna().sum()

0

---

### 3. feature: longtitude

**Observation:**
- zero places the location in the Atlantic Ocean

**Action:** 
- impute missing values with median

In [55]:
df['longitude'].replace(0, np.nan, inplace=True)

for status in df['status_group'].unique():
    df = impute_nan_by_status_group(feature_name = 'longitude', status=status, aggregation_method='median')

Replaced nulls with 35.08636786 for functional
Replaced nulls with 35.048525645 for non functional
Replaced nulls with 34.18320909 for functional needs repair


In [39]:
# sanity check
df['longitude'].isna().sum()

0

---

### 4. feature: population

**Observation:** <br>
- either noone lives near the pump, or it is missing value
<br>

**Assumption:** <br>
- zeroes represent missing value 
<br>

**Action:**
- impute missing values with median

In [40]:
df['population'].replace(0, np.nan, inplace=True)
for status in df['status_group'].unique():
    df = impute_nan_by_status_group(feature_name = 'population', status=status, aggregation_method='median')

Replaced nulls with 150.0 for functional
Replaced nulls with 150.0 for non functional
Replaced nulls with 200.0 for functional needs repair


In [41]:
# sanity check
df['population'].isna().sum()

0

### 5. feature: contruction_year

**Observation:** <br>
- construction_year cannot be zero, so it is definitely a missing value
<br>

**Action:**
- impute missing values with median

In [42]:
df['construction_year'].replace(0, np.nan, inplace=True)
for status in df['status_group'].unique():
    df = impute_nan_by_status_group(feature_name = 'construction_year', status=status, aggregation_method='median')

Replaced nulls with 2003.0 for functional
Replaced nulls with 1994.0 for non functional
Replaced nulls with 1998.0 for functional needs repair


In [43]:
# sanity check
df['construction_year'].isna().sum()

0

<span style ="color:red"> Important consideration after imputing missing data based on segments defined by the target variable </span>
1. Information Leakage: We performed imputation by segmentation on all observations with missing values before splitting to training and testing subsets. This might lead to overoptimistic estimates of model's performance
2. Bias and Variance: Could introduce bias, if imputation alters the underlying distribution of the feature. On the flipside, it could also reduce variance
3. Alternative Imputation Methods: K-nearest neigbors imputation, or other model-based imputation such as random forest to predict missing values

---

In [44]:
# get a list of numerical features
non_numerical_features = df.drop(columns = ['status_group'])\
                            .select_dtypes(include=['category', 'object', 'datetime64[ns]'])\
                                .columns\
                                    .to_list()

# get a series of non numerical columns with missing values
columns_with_nas: pd.Series = df[non_numerical_features].apply(lambda col: (col.isna()).sum())

# Display columns with a significant number of missing values
print("Absolute Number:")
print(columns_with_nas[columns_with_nas > 0])

print("\nIn PERCENTAGE %:")
print(columns_with_nas[columns_with_nas > 0]/len(df) * 100)

Absolute Number:
funder                3637
installer             3655
wpt_name                 2
subvillage             371
public_meeting        3334
scheme_management     3878
scheme_name          28810
permit                3056
dtype: int64

In PERCENTAGE %:
funder                6.122896
installer             6.153199
wpt_name              0.003367
subvillage            0.624579
public_meeting        5.612795
scheme_management     6.528620
scheme_name          48.501684
permit                5.144781
dtype: float64


**Observation:** <br>
- scheme_name: with > 48% data missing, imputing might introduce a lot of bias. Lets drop it
- scheme_management, installer, funder: ~ 6% data missing. We can impute with mode(most frequent value) or a placeholder like "unknown"
- public_meeting and permit: ~ 5% missing. Boolean features. Can treat missing value as third category "unknown"
- subvillage: ~ 0.62% missing. Impute mode or replace with "unknown"
- wpt_name: ~0.003% missing. Impute with mode or drop rows completely

In [45]:
df['scheme_name'] = df['scheme_name'].cat.add_categories(['Unknown'])
df['scheme_name'].fillna('Unknown', inplace=True)

In [46]:
for column in ['scheme_management', 'installer', 'funder']:
    most_frequent_value = df[column].mode()[0]
    df[column].fillna(most_frequent_value, inplace=True)

In [47]:
for column in ['public_meeting', 'permit']:
    df[column].fillna('Unknown', inplace=True)

In [48]:
most_frequent_subvillage = df['subvillage'].mode()[0]
df['subvillage'].fillna(most_frequent_subvillage, inplace=True)


In [49]:
most_frequent_wpt_name = df['wpt_name'].mode()[0]
df['wpt_name'].fillna(most_frequent_wpt_name, inplace=True)


---

In [52]:
missing_values_after = df.isnull().sum()
print(missing_values_after[missing_values_after > 0])

Series([], dtype: int64)


---

Export modifed dataset

In [53]:
# Specify the file path where you want to save the DataFrame
file_path = '../data/processed/post_missing_values_data.csv'

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)